# Notes on Manabe/Wetherald 1967

## Introduction

- It's a follow up study
- Preceded by papers:
   - in 1961 (Manabe & Möller): radiative convective equilibrium?
   - 1964 (Manabe/Strickler): thermal equilibrium of the atmosphere with convective adjustment
   - 1965 (Manabe et al): Use computation (numerical integration) to model radiative transfer in the atmosphere including the hydrologic cycle
      - Used a climatological distribution of absolute humidity as approximation
- Next step (this paper): numerically solve the fully coupled radiative transfer and hydrological cycle elements.
- But before doing so, compute a few radiative convective equilibria with fixed relative humidity to answer:
  1) How fast does the atmosphere equilibriate when RH is realistic but time-invariant?
  2) How do optical/radiative variables affect the atmosphere's equilibrium temperature under realistic but static RH conditions?
  3) From the atmosphere temperature calculated in 1) and 2), what's the surface equilibrium temp?
- Use those answers to sanity check more complex models

- Möller 1963 discussed CO2 content vs. magnitude of LWIR at earth's surface relating to equilibrium temperature
  - Relative vs. absolute humidity confusion
  - Similar order-of-magnitude dependence of equilibrium temperature on CO2 content vs papers from the past 15 years
  - "We should probably re-evaluate"
